# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2022 9:13AM,239725,10,8151846,Beach Products Inc,Released
1,Jun 16 2022 9:13AM,239725,10,8151847,Beach Products Inc,Released
2,Jun 16 2022 9:13AM,239725,10,8151848,Beach Products Inc,Released
3,Jun 16 2022 9:13AM,239725,10,8151849,Beach Products Inc,Released
4,Jun 16 2022 9:13AM,239725,10,8151850,Beach Products Inc,Released
5,Jun 16 2022 9:13AM,239725,10,8151851,Beach Products Inc,Released
6,Jun 16 2022 9:13AM,239725,10,8151852,Beach Products Inc,Released
7,Jun 16 2022 9:13AM,239725,10,8151853,Beach Products Inc,Released
8,Jun 16 2022 9:13AM,239725,10,8151854,Beach Products Inc,Released
9,Jun 16 2022 9:13AM,239725,10,8151855,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,239716,Released,29
40,239717,Released,17
41,239720,Released,4
42,239721,Released,2
43,239725,Released,20


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239716,NaN,NaN,29.0
239717,NaN,NaN,17.0
239720,NaN,NaN,4.0
239721,NaN,NaN,2.0
239725,NaN,NaN,20.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239629,0.0,0.0,1.0
239641,21.0,5.0,0.0
239642,0.0,0.0,1.0
239643,0.0,0.0,1.0
239654,0.0,72.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239629,0,0,1
239641,21,5,0
239642,0,0,1
239643,0,0,1
239654,0,72,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239629,0,0,1
1,239641,21,5,0
2,239642,0,0,1
3,239643,0,0,1
4,239654,0,72,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239629,,,1
1,239641,21,5,
2,239642,,,1
3,239643,,,1
4,239654,,72,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2022 9:13AM,239725,10,Beach Products Inc
17,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance
20,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc."
22,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc."
26,Jun 16 2022 8:58AM,239717,20,Alumier Labs Inc.
43,Jun 16 2022 8:55AM,239716,10,ISDIN Corporation
72,Jun 16 2022 8:55AM,239715,10,ISDIN Corporation
115,Jun 16 2022 8:36AM,239712,10,ISDIN Corporation
127,Jun 16 2022 8:31AM,239711,10,Emerginnova
129,Jun 16 2022 7:55AM,239708,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 16 2022 9:13AM,239725,10,Beach Products Inc,,,20
1,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance,,,20
2,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc.",,,2
3,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc.",,,4
4,Jun 16 2022 8:58AM,239717,20,Alumier Labs Inc.,,,17
5,Jun 16 2022 8:55AM,239716,10,ISDIN Corporation,,,29
6,Jun 16 2022 8:55AM,239715,10,ISDIN Corporation,,,43
7,Jun 16 2022 8:36AM,239712,10,ISDIN Corporation,,,12
8,Jun 16 2022 8:31AM,239711,10,Emerginnova,,,2
9,Jun 16 2022 7:55AM,239708,12,Hush Hush,,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 9:13AM,239725,10,Beach Products Inc,20,,
1,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance,20,,
2,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc.",2,,
3,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc.",4,,
4,Jun 16 2022 8:58AM,239717,20,Alumier Labs Inc.,17,,
5,Jun 16 2022 8:55AM,239716,10,ISDIN Corporation,29,,
6,Jun 16 2022 8:55AM,239715,10,ISDIN Corporation,43,,
7,Jun 16 2022 8:36AM,239712,10,ISDIN Corporation,12,,
8,Jun 16 2022 8:31AM,239711,10,Emerginnova,2,,
9,Jun 16 2022 7:55AM,239708,12,Hush Hush,10,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 9:13AM,239725,10,Beach Products Inc,20,,
1,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance,20,,
2,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc.",2,,
3,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc.",4,,
4,Jun 16 2022 8:58AM,239717,20,Alumier Labs Inc.,17,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 9:13AM,239725,10,Beach Products Inc,20.0,NaN,NaN
1,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance,20.0,NaN,NaN
2,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc.",2.0,NaN,NaN
3,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc.",4.0,NaN,NaN
4,Jun 16 2022 8:58AM,239717,20,Alumier Labs Inc.,17.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 9:13AM,239725,10,Beach Products Inc,20.0,0.0,0.0
1,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance,20.0,0.0,0.0
2,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc.",2.0,0.0,0.0
3,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc.",4.0,0.0,0.0
4,Jun 16 2022 8:58AM,239717,20,Alumier Labs Inc.,17.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4074800,191.0,3.0,10.0
12,719102,15.0,0.0,0.0
16,239699,1.0,0.0,0.0
18,718957,3.0,0.0,0.0
19,719073,26.0,18.0,0.0
20,2396828,56.0,79.0,21.0
22,719015,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4074800,191.0,3.0,10.0
1,12,719102,15.0,0.0,0.0
2,16,239699,1.0,0.0,0.0
3,18,718957,3.0,0.0,0.0
4,19,719073,26.0,18.0,0.0
5,20,2396828,56.0,79.0,21.0
6,22,719015,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,191.0,3.0,10.0
1,12,15.0,0.0,0.0
2,16,1.0,0.0,0.0
3,18,3.0,0.0,0.0
4,19,26.0,18.0,0.0
5,20,56.0,79.0,21.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,191.0
1,12,Released,15.0
2,16,Released,1.0
3,18,Released,3.0
4,19,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,22
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,21.0,0.0
Executing,3.0,0.0,0.0,0.0,18.0,79.0,0.0
Released,191.0,15.0,1.0,3.0,26.0,56.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,22
0,Completed,10.0,0.0,0.0,0.0,0.0,21.0,0.0
1,Executing,3.0,0.0,0.0,0.0,18.0,79.0,0.0
2,Released,191.0,15.0,1.0,3.0,26.0,56.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,22
0,Completed,10.0,0.0,0.0,0.0,0.0,21.0,0.0
1,Executing,3.0,0.0,0.0,0.0,18.0,79.0,0.0
2,Released,191.0,15.0,1.0,3.0,26.0,56.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()